In [8]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime

# Define the dropdown widgets for input
start_year = widgets.Dropdown(
    options=[str(year) for year in range(2000, 2025)],
    description='Start Year:'
)
start_month = widgets.Dropdown(
    options=[f"{month:02d}" for month in range(1, 13)],
    description='Start Month:'
)
start_day = widgets.Dropdown(
    options=[f"{day:02d}" for day in range(1, 32)],
    description='Start Day:'
)

end_year = widgets.Dropdown(
    options=[str(year) for year in range(2000, 2025)],
    description='End Year:'
)
end_month = widgets.Dropdown(
    options=[f"{month:02d}" for month in range(1, 13)],
    description='End Month:'
)
end_day = widgets.Dropdown(
    options=[f"{day:02d}" for day in range(1, 32)],
    description='End Day:'
)

aoi_input = widgets.Text(
    description="AOI (bbox):",
    placeholder="Enter bbox as minX,minY,maxX,maxY"
)

# Display the widgets
display(start_year, start_month, start_day, end_year, end_month, end_day, aoi_input)

# Function to retrieve input from widgets
# Function to retrieve input from widgets
def get_inputs():
    try:
        # Validate dates
        start_date = datetime(
            int(start_year.value), int(start_month.value), int(start_day.value)
        ).strftime('%Y-%m-%d')
        end_date = datetime(
            int(end_year.value), int(end_month.value), int(end_day.value)
        ).strftime('%Y-%m-%d')
        
        # Validate AOI input
        try:
            aoi = [float(coord) for coord in aoi_input.value.split(",")]
            if len(aoi) != 4:
                print("Invalid AOI format. Please enter as minX,minY,maxX,maxY.")
                return None, None, None
            # Validate bounding box coordinates
            if not (aoi[0] < aoi[2] and aoi[1] < aoi[3]):
                print("Invalid AOI: Ensure minX < maxX and minY < maxY.")
                return None, None, None
        except ValueError as e:
            print(f"Invalid AOI input. Error: {e}")
            return None, None, None
        
        return start_date, end_date, aoi
    except ValueError as e:
        print(f"Input error: {e}")
        return None, None, None

# List satellite collections with metadata
satellite_sources = {
    "Sentinel-2": {
        "Temporal Extent": "2015-Present",
        "Spatial Resolution": "10m (Visible, NIR), 20m (Red Edge, SWIR), 60m (Coastal, Aerosols)",
        "Return Time": "5 days",
        "Bands": "B02 (Blue), B03 (Green), B04 (Red), B08 (NIR), others"
    },
    "Landsat 8-9": {
        "Temporal Extent": "2013-Present",
        "Spatial Resolution": "30m",
        "Return Time": "16 days",
        "Bands": "B1 (Coastal), B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), others"
    },

    "NAIP (US only)": {
        "Temporal Extent": "2010-Present (varies by state)",
        "Spatial Resolution": "0.3m-1m",
        "Return Time": "Every 2-3 years",
        "Bands": "R, G, B, NIR (varies)"
    },
}

# Print satellite source options and metadata
print("\nAvailable Satellite Data Sources for Vegetation Analysis:\n")
for source, metadata in satellite_sources.items():
    print(f"{source}:")
    for key, value in metadata.items():
        print(f"  {key}: {value}")
    print()

Dropdown(description='Start Year:', options=('2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '…

Dropdown(description='Start Month:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11'…

Dropdown(description='Start Day:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', …

Dropdown(description='End Year:', options=('2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '20…

Dropdown(description='End Month:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', …

Dropdown(description='End Day:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '1…

Text(value='', description='AOI (bbox):', placeholder='Enter bbox as minX,minY,maxX,maxY')


Available Satellite Data Sources for Vegetation Analysis:

Sentinel-2:
  Temporal Extent: 2015-Present
  Spatial Resolution: 10m (Visible, NIR), 20m (Red Edge, SWIR), 60m (Coastal, Aerosols)
  Return Time: 5 days
  Bands: B02 (Blue), B03 (Green), B04 (Red), B08 (NIR), others

Landsat 8-9:
  Temporal Extent: 2013-Present
  Spatial Resolution: 30m
  Return Time: 16 days
  Bands: B1 (Coastal), B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), others

NAIP (US only):
  Temporal Extent: 2010-Present (varies by state)
  Spatial Resolution: 0.3m-1m
  Return Time: Every 2-3 years
  Bands: R, G, B, NIR (varies)



In [10]:
import ipywidgets as widgets
from shapely.geometry import box
import geopandas as gpd
import pystac_client

# Function to query satellite collections using the previously set widget values
def query_collections():
    # Retrieve the values from the widgets
    start_date = f"{start_year.value}-{start_month.value}-{start_day.value}"
    end_date = f"{end_year.value}-{end_month.value}-{end_day.value}"
    aoi = [float(coord) for coord in aoi_input.value.split(",")]

    # Validate AOI format
    if len(aoi) != 4:
        raise ValueError("Invalid AOI format. Please enter as minX,minY,maxX,maxY.")
    
    # Ensure AOI is a valid bounding box
    try:
        aoi_geometry = box(*aoi)  # This will raise an error if coordinates are invalid
        bbox = aoi_geometry.bounds  # Get bounding box coordinates
    except Exception as e:
        raise ValueError(f"Error creating bounding box from AOI: {e}")

    # MPC STAC API endpoint
    stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
    catalog = pystac_client.Client.open(stac_url)

    # Satellite collections and their STAC IDs
    collections = {
        "Sentinel-2": "sentinel-2-l2a",
        "Landsat 8-9": "landsat-8-c2-l2",
        "NAIP (US only)": "naip",
    }

    # Results list to store collection data
    all_results = []

    # Query each collection
    for name, collection_id in collections.items():
        collection_box = widgets.VBox()  # Create a VBox for each collection's widget box
        try:
            search = catalog.search(
                collections=[collection_id],
                datetime=f"{start_date}/{end_date}",
                bbox=bbox
            )
            items = list(search.get_items())
            if items:
                # Count images
                num_images = len(items)
                
                # Initialize default values for cloud coverage and dates
                avg_cloud_coverage = "N/A"
                non_cloudy_images = 0
                non_cloudy_dates = []

                # If not NAIP, calculate cloud coverage and filter non-cloudy images
                if name != "NAIP (US only)":
                    cloud_coverages = [
                        float(item.properties.get("eo:cloud_cover", 0)) for item in items if "eo:cloud_cover" in item.properties
                    ]
                    avg_cloud_coverage = sum(cloud_coverages) / len(cloud_coverages) if cloud_coverages else "N/A"
                    
                    # Filter non-cloudy images (less than 5% cloud cover)
                    non_cloudy_items = [
                        item for item in items 
                        if float(item.properties.get("eo:cloud_cover", 100)) < 5
                    ]
                    non_cloudy_images = len(non_cloudy_items)

                # Add the results as text to a text widget
                results_text = (
                    f"**Collection:** {name}\n"
                    f"**Number of Images:** {num_images}\n"
                    f"**Average Cloud Coverage:** {avg_cloud_coverage}\n"
                    f"**Non-Cloudy Images (<5% Cloud):** {non_cloudy_images}\n"
                )
                results_widget = widgets.Textarea(
                    value=results_text,
                    layout=widgets.Layout(width="90%", height="100px"),
                    disabled=True
                )

                # For non-NAIP collections, display the results without the non-cloudy date list
                collection_box.children = [results_widget]

            else:
                # Handle no items case
                results_text = (
                    f"**Collection:** {name}\n"
                    f"**Number of Images:** 0\n"
                    f"**Average Cloud Coverage:** N/A\n"
                    f"**Non-Cloudy Images (<5% Cloud):** 0\n"
                )
                results_widget = widgets.Textarea(
                    value=results_text,
                    layout=widgets.Layout(width="90%", height="100px"),
                    disabled=True
                )

                # Add the results widget to the collection's VBox
                collection_box.children = [results_widget]

            # Add the collection's widget box to the main results list
            all_results.append(collection_box)

        except Exception as e:
            # Handle error case
            error_text = f"**Collection:** {name}\nError retrieving data: {e}\n"
            error_widget = widgets.Textarea(
                value=error_text,
                layout=widgets.Layout(width="90%", height="100px"),
                disabled=True
            )
            collection_box.children = [error_widget]
            all_results.append(collection_box)

    # Display all collection widgets
    display(*all_results)

# Call the query function immediately after defining it to load the results with widget inputs
query_collections()


In [5]:
import pystac_client
import rasterio
import numpy as np
from shapely.geometry import box
from rasterio.mask import mask
import ipywidgets as widgets
import geopandas as gpd
import os

# Function to query satellite collections and mosaic the images (all bands)
def mosaic_images():
    # Retrieve the values from the widgets
    start_date = f"{start_year.value}-{start_month.value}-{start_day.value}"
    end_date = f"{end_year.value}-{end_month.value}-{end_day.value}"
    aoi = [float(coord) for coord in aoi_input.value.split(",")]

    # Validate AOI format
    if len(aoi) != 4:
        raise ValueError("Invalid AOI format. Please enter as minX,minY,maxX,maxY.")
    
    # Ensure AOI is a valid bounding box
    try:
        aoi_geometry = box(*aoi)  # This will raise an error if coordinates are invalid
        bbox = aoi_geometry.bounds  # Get bounding box coordinates
    except Exception as e:
        raise ValueError(f"Error creating bounding box from AOI: {e}")

    # MPC STAC API endpoint
    stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
    catalog = pystac_client.Client.open(stac_url)

    # Satellite collections and their STAC IDs
    collections = {
        "Sentinel-2": "sentinel-2-l2a",
        "Landsat 8-9": "landsat-8-c2-l2",
        "NAIP (US only)": "naip",
    }

    # Loop over each collection
    for name, collection_id in collections.items():
        try:
            search = catalog.search(
                collections=[collection_id],
                datetime=f"{start_date}/{end_date}",
                bbox=bbox
            )
            items = list(search.get_items())

            # Filter out images with low cloud coverage for Sentinel-2 and Landsat
            if name != "NAIP (US only)":
                low_cloud_items = [
                    item for item in items 
                    if float(item.properties.get("eo:cloud_cover", 100)) < 5
                ]
            else:
                low_cloud_items = items  # For NAIP, no cloud filtering

            # Process images (mosaic them)
            if low_cloud_items:
                # Open and mosaic the images
                src_list = []
                all_bands = []  # To store the bands for each image

                for item in low_cloud_items:
                    # Get the asset URLs for all bands
                    asset_urls = {key: item.assets[key].href for key in item.assets if key.startswith('B')}
                    
                    # For each band in the asset, read and process it
                    band_arrays = []
                    for band_key, band_url in asset_urls.items():
                        with rasterio.open(band_url) as src:
                            # Mask the image with the AOI
                            out_image, out_transform = mask(src, [aoi_geometry], crop=True)
                            band_arrays.append(out_image[0])  # Only the first band for each image (each asset is single-band)
                        
                    # Stack all the bands from this image
                    all_bands.append(np.stack(band_arrays, axis=0))

                # Check if there are any images to mosaic
                if all_bands:
                    # Stack the images into a 4D array (num_images x num_bands x height x width)
                    stacked_images = np.stack(all_bands)

                    # Mosaic images (average to get a simple mosaic for each band)
                    mosaic_image = np.mean(stacked_images, axis=0)

                    # Write the mosaic to a new file
                    output_file = f"{name}_multi_band_mosaic.tif"
                    with rasterio.open(
                        output_file, "w", 
                        driver="GTiff", 
                        count=mosaic_image.shape[0],  # Number of bands
                        dtype=mosaic_image.dtype, 
                        crs=src.crs, 
                        transform=out_transform, 
                        width=mosaic_image.shape[2], 
                        height=mosaic_image.shape[1]
                    ) as dest:
                        for i in range(mosaic_image.shape[0]):
                            dest.write(mosaic_image[i], i+1)  # Write each band separately
                    print(f"Multi-band mosaic for {name} saved as {output_file}")
                else:
                    print(f"No valid images for {name} in the given date range.")
            else:
                print(f"No images with low cloud cover for {name}.")
                
        except Exception as e:
            print(f"Error processing collection {name}: {e}")

# Call the mosaic function to generate the mosaic for each collection
mosaic_images()

Error processing collection Sentinel-2: HTTP response code: 404 - Recv failure: Connection reset by peer
Error processing collection Landsat 8-9: need at least one array to stack
Error processing collection NAIP (US only): need at least one array to stack
